# Google Analytics

O ***Google Analytics*** é a ferramenta de Web Analytics do Google. Com ela é possível mensurar o comportamento de usuários em sites e aplicativos.

Para ajudar no treinamento da ferramenta, o Google oferece acesso a conta do Google Analytics da sua loja virtual de brindes.

* Site: shop.googlemerchandisestore.com/

* Conta Analytics Demo: https://analytics.google.com/analytics/web/demoAccount

Caso você queira entender um pouco mais sobre a conta demo do Google Analytics, existe uma sessão no suporte do Google sobre o tema: https://support.google.com/analytics/answer/6367342?hl=pt-BR#access

# Formato de Dados
Para desenvolvermos um algoritmo capaz de prever o comportamento de usuários no site, primeiro precisamos entender o formato dos dados que vamos trabalhar.

Nesse caso, vamos enteder 2 conceitos do Google Analytics que vão ser essenciais para o desenvolvimento desse curso:

* ***Visitante***: o campo fullVisitorId representa um visitante único no site. Caso um usuário acesse o site mais de uma vez, atribuir o mesmo fullVisitorId depende da capacidade do Google de identificar o visitante. Se o mesmo usuário acessa o site de diferentes dispositivos ou faz uma limpeza de cookies, ele pode ser considerado um visitante novo no site. Assim como pessoas diferentes que acessem o site do mesmo dispositivo podem ser consideradas o mesmo fullVisitorId.

* ***Visita***: também conhecida como sessão, é identificada na nossa base de dados pelo visitId. Uma sessão é um grupo de interações do usuário com o site que ocorrem dentro de um determinado período de tempo. Um único usuário pode abrir várias sessões. Essas sessões podem ocorrer no mesmo dia ou em vários dias, semanas ou meses.

O entendimento desses dois conceitos é importante por que cada linha do dataset utilizado nesse curso representa uma sessão. Sendo assim, podemos ter o mesmo usuário em linhas diferentes, com um visitId diferente.

![data_model](https://www.bounteous.com/sites/default/files/luna-migrate/BQ-Rows-2-e1462998313180.png)

## Variáveis
Caso queira entender melhor as informações de cada variável utilizada nesse curso, o Google Analytics oferece um descritivo de todas as colunas que são possíveis ser extraídas do nesse link: https://support.google.com/analytics/answer/3437719?hl=pt-BR

# Preparação dos Dados

In [71]:
import pandas as pd

Definição de uma variável com a query que irá executar no Google Big Query

In [72]:
query = 'SELECT * EXCEPT(hits,customDimensions) FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` WHERE date BETWEEN "20170301" and "20170331"'

In [73]:
df = pd.read_gbq(query,project_id='seu projeto aqui')

Requesting query... ok.
Job ID: a470d95b-cd77-4841-87a7-4aaaa939a971
Query running...
Query done.
Processed: 728.8 MB Billed: 729.0 MB
Standard price: $0.00 USD

Retrieving results...
Got 69931 rows.

Total time taken 59.4 s.
Finished at 2019-03-24 12:00:29.


Observando as primeiras colunas da base de dados, podemos notar que algumas colunas contém informações no tipo chave-valor, ou conhecido em Python como dicionários. Por exemplo: geoNetwork, totals, etc...

In [74]:
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,None,1,1490578210,1490578210,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': True, 'medium': '(none)', 'so...","{'operatingSystem': 'Windows', 'browser': 'Int...","{'latitude': 'not available in demo dataset', ...",7376642514595539072,None,None,Direct,Not Socially Engaged
1,None,1,1490567601,1490567601,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': None, 'medium': 'organic', 's...","{'operatingSystem': 'Macintosh', 'browser': 'C...","{'latitude': 'not available in demo dataset', ...",9626273823699824058,None,None,Organic Search,Not Socially Engaged
2,None,1,1490555169,1490555169,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': None, 'medium': 'organic', 's...","{'operatingSystem': 'Windows', 'browser': 'Chr...","{'latitude': 'not available in demo dataset', ...",1942902880160520154,None,None,Organic Search,Not Socially Engaged
3,None,1,1490532267,1490532267,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': None, 'medium': 'referral', '...","{'operatingSystem': 'iOS', 'browser': 'Safari'...","{'latitude': 'not available in demo dataset', ...",8992277135232148365,None,None,Referral,Not Socially Engaged
4,None,1,1490540259,1490540259,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': True, 'medium': '(none)', 'so...","{'operatingSystem': 'Android', 'browser': 'Chr...","{'latitude': 'not available in demo dataset', ...",7072308722185366175,None,None,Direct,Not Socially Engaged


In [75]:
df.dtypes

visitorId               object
visitNumber              int64
visitId                  int64
visitStartTime           int64
date                    object
totals                  object
trafficSource           object
device                  object
geoNetwork              object
fullVisitorId           object
userId                  object
clientId                object
channelGrouping         object
socialEngagementType    object
dtype: object

In [ ]:
df.totals[0]

Podemos considerar que cada "chave"(Ex: "browser", "browserVersion", "deviceCategory") dentro dessa coluna, é uma informação a mais sobre essa linha. Sendo assim, para facilitar a nossa manipulação e treino de modelo, podemos converter cada valor dentro dessas "chaves" para uma coluna específica.

Primeiramente, vamos identificar as colunas que contém dicionários:

In [76]:
dicionarios = ['device','geoNetwork','trafficSource','totals']

Nesta parte do código, para cada coluna que definimos na variável dicionário, vamos carregar cada uma das linhas em uma lista e passar para a função DataFrame converter em um Series.

Após isso basta unir o nosso DataFrame original(df) ao novo DataFrame gerado através da função "join".

In [77]:
for coluna in dicionarios:
    df = df.join(pd.DataFrame([linha for linha in df[coluna]]))

In [78]:
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,fullVisitorId,...,pageviews,screenviews,sessionQualityDim,timeOnScreen,timeOnSite,totalTransactionRevenue,transactionRevenue,transactions,uniqueScreenviews,visits
0,None,1,1490578210,1490578210,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': True, 'medium': '(none)', 'so...","{'operatingSystem': 'Windows', 'browser': 'Int...","{'latitude': 'not available in demo dataset', ...",7376642514595539072,...,1.0,None,None,None,NaN,NaN,NaN,NaN,None,1
1,None,1,1490567601,1490567601,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': None, 'medium': 'organic', 's...","{'operatingSystem': 'Macintosh', 'browser': 'C...","{'latitude': 'not available in demo dataset', ...",9626273823699824058,...,1.0,None,None,None,NaN,NaN,NaN,NaN,None,1
2,None,1,1490555169,1490555169,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': None, 'medium': 'organic', 's...","{'operatingSystem': 'Windows', 'browser': 'Chr...","{'latitude': 'not available in demo dataset', ...",1942902880160520154,...,1.0,None,None,None,NaN,NaN,NaN,NaN,None,1
3,None,1,1490532267,1490532267,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': None, 'medium': 'referral', '...","{'operatingSystem': 'iOS', 'browser': 'Safari'...","{'latitude': 'not available in demo dataset', ...",8992277135232148365,...,1.0,None,None,None,NaN,NaN,NaN,NaN,None,1
4,None,1,1490540259,1490540259,20170326,"{'timeOnSite': None, 'sessionQualityDim': None...","{'isTrueDirect': True, 'medium': '(none)', 'so...","{'operatingSystem': 'Android', 'browser': 'Chr...","{'latitude': 'not available in demo dataset', ...",7072308722185366175,...,1.0,None,None,None,NaN,NaN,NaN,NaN,None,1


Para criação das variáveis, vamos salvar as chaves da coluna "totals" dentro de uma lista.

In [79]:
totals = df.totals[0].keys()

In [80]:
totals = list(totals)

Após finalizarmos a transformação podemos excluir as colunas originais dos dicionários:

In [81]:
df.drop(dicionarios, axis=1, inplace=True)

# Feature Engineering

Para garantir que todas quantitativas estejam identificadas corretamente, vamos usar a função to_numeric em cada uma.

In [82]:
for coluna in totals:
    df[coluna] = pd.to_numeric(df[coluna])

In [83]:
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType,...,pageviews,screenviews,sessionQualityDim,timeOnScreen,timeOnSite,totalTransactionRevenue,transactionRevenue,transactions,uniqueScreenviews,visits
0,None,1,1490578210,1490578210,20170326,7376642514595539072,None,None,Direct,Not Socially Engaged,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,None,1,1490567601,1490567601,20170326,9626273823699824058,None,None,Organic Search,Not Socially Engaged,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,None,1,1490555169,1490555169,20170326,1942902880160520154,None,None,Organic Search,Not Socially Engaged,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,None,1,1490532267,1490532267,20170326,8992277135232148365,None,None,Referral,Not Socially Engaged,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,None,1,1490540259,1490540259,20170326,7072308722185366175,None,None,Direct,Not Socially Engaged,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Segundo a documentação do Google Analytics, o valor do Revenue é multiplicado por 1.000.000. Para trabalhar com o número real, vamos dividir essa colunas por 1.000.000.

https://support.google.com/analytics/answer/3437719?hl=pt-BR

In [84]:
df.totalTransactionRevenue = df.totalTransactionRevenue / 1000000

Aqui chegamos na parte principal da criação de modelos de machine learning com esse tipo de dados. O Dataset possui mais de uma linha por usuário, e o nosso desafio é prever o valor total gasto por usuário. 

Para conseguir treinar um algoritmo de machine learning nesse caso, primeiro precisamos agregar as informações por usuário e criar as variáveis preditoras baseando no conhecimento que temos sobre a base de dados. 

A nosssa primeira estratégia será treinar um algoritmo baseado nas informações quantitativas dos usuários

In [85]:
df_totals = df.groupby('fullVisitorId',as_index=False)[totals].sum()

In [86]:
df_totals.head()

,fullVisitorId,timeOnSite,sessionQualityDim,totalTransactionRevenue,pageviews,visits,hits,newVisits,timeOnScreen,uniqueScreenviews,transactionRevenue,screenviews,transactions,bounces
0,0000039460501403861,99.0,0.0,0.0,2.0,1,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000530033766739584,556.0,0.0,0.0,4.0,1,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000702913088027926,284.0,0.0,0.0,10.0,6,10,1.0,0.0,0.0,0.0,0.0,0.0,4.0
3,0000734968258259612,0.0,0.0,0.0,1.0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0001413363431757398,40.0,0.0,0.0,2.0,1,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
df_totals.shape

(57888, 14)

# Treinando o Modelo

Agora vamos copiar a variável resposta para um variável "y".

In [88]:
y = df_totals.totalTransactionRevenue.copy()

In [89]:
X = df_totals.drop(['transactionRevenue','totalTransactionRevenue'],axis=1)

Utilizaremos o método train_test_split do sklearn para dividir a base em treino e teste.

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [92]:
X_train.head()

,fullVisitorId,timeOnSite,sessionQualityDim,pageviews,visits,hits,newVisits,timeOnScreen,uniqueScreenviews,screenviews,transactions,bounces
49072,8456607167663347373,284.0,0.0,8.0,2,8,1.0,0.0,0.0,0.0,0.0,1.0
3589,0611946115451867044,0.0,0.0,1.0,1,1,1.0,0.0,0.0,0.0,0.0,1.0
44093,7605802868055219233,54.0,0.0,2.0,1,2,1.0,0.0,0.0,0.0,0.0,0.0
32918,5687465682644926905,0.0,0.0,1.0,1,1,1.0,0.0,0.0,0.0,0.0,1.0
6663,1146222293825551946,0.0,0.0,1.0,1,1,1.0,0.0,0.0,0.0,0.0,1.0


In [93]:
y_train.head()

49072    0.0
3589     0.0
44093    0.0
32918    0.0
6663     0.0
Name: totalTransactionRevenue, dtype: float64

Agora vamos tentar prever o valor gasto por usuário utilizando uma regressão linear.

In [94]:
from sklearn.linear_model import LinearRegression

In [95]:
reg = LinearRegression()

In [96]:
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [97]:
reg_predict = reg.predict(X_test)

In [98]:
reg_predict

array([2.70935773, 2.68813456, 2.91068411, ..., 2.82231989, 2.87297002,
       2.71276789])

# Avaliando a performance

In [99]:
resultados = pd.DataFrame()

In [100]:
resultados = pd.DataFrame()
resultados['revenue'] = y_test
resultados['predict'] = reg_predict
resultados['erro'] = reg_predict - y_test

In [101]:
resultados.head()

,revenue,predict,erro
33733,0.0,2.709358,2.709358
37120,0.0,2.688135,2.688135
1227,0.0,2.910684,2.910684
46944,0.0,2.627949,2.627949
31702,0.0,2.898701,2.898701


In [102]:
resultados[resultados.revenue > 0].head()

,revenue,predict,erro
43035,85.08,2.651793,-82.428207
39840,117.97,2.671493,-115.298507
31853,116.60,2.720783,-113.879217
57216,37.97,2.563864,-35.406136
33173,74.96,2.712838,-72.247162


Para avaliar a performance do nosso modelo vamos utilizar a raiz do erro médio quadrático.

In [103]:
import numpy as np

In [104]:
np.sqrt(np.mean((reg_predict - y_test)**2))

35.34186409283866

Para tentar melhorar a performance do modelo, vamos tentar utilizar um outro algoritmo, no caso o GradientBoosting da biblioteca ScikitLearn.

In [105]:
from sklearn.ensemble import GradientBoostingRegressor

In [106]:
gb = GradientBoostingRegressor(random_state=42)

In [107]:
gb.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=1.0, verbose=0, warm_start=False)

In [108]:
gb_predict = gb.predict(X_test)

In [109]:
gb_predict[gb_predict < 0 ] = 0

In [110]:
resultados = pd.DataFrame()
resultados['revenue'] = y_test
resultados['predict'] = gb_predict
resultados['erro'] = gb_predict - y_test
resultados.head()

,revenue,predict,erro
33733,0.0,0.0,0.0
37120,0.0,0.0,0.0
1227,0.0,0.0,0.0
46944,0.0,0.0,0.0
31702,0.0,0.0,0.0


In [111]:
resultados[resultados.revenue > 0]

,revenue,predict,erro
43035,85.08,119.562100,34.482100
39840,117.97,208.387892,90.417892
31853,116.60,97.415404,-19.184596
57216,37.97,97.415404,59.445404
33173,74.96,206.650972,131.690972
3173,34.98,97.415404,62.435404
11209,196.30,97.415404,-98.884596
4172,32.15,94.039649,61.889649
47601,24.98,97.415404,72.435404
12495,56.99,97.415404,40.425404


In [112]:
np.sqrt(mean_squared_error(y_test,gb_predict))

32.429814072226705